<a href="https://colab.research.google.com/github/xx529/NLP/blob/main/ESIM/%E8%9A%82%E8%9A%81%E9%87%91%E8%9E%8D%E8%AF%AD%E4%B9%89%E7%9B%B8%E4%BC%BC%E5%BA%A6%E4%BB%BB%E5%8A%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# !pip install pytorch_lightning

     |████████████████████████████████| 915 kB 4.0 MB/s 
     |████████████████████████████████| 234 kB 12.5 MB/s 
     |████████████████████████████████| 829 kB 14.1 MB/s 
     |████████████████████████████████| 118 kB 19.4 MB/s 
     |████████████████████████████████| 636 kB 18.8 MB/s 
     |████████████████████████████████| 10.6 MB 22.5 MB/s 
     |████████████████████████████████| 1.3 MB 40.7 MB/s 
     |████████████████████████████████| 142 kB 55.1 MB/s 
     |████████████████████████████████| 294 kB 46.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=e3962e88ca5bf3b9e504488272cb842ce2cb360990094cdb6361b889bf708bf7
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstal

In [6]:
import pandas as pd
from tqdm import tqdm
import json
import bz2
import numpy as np
import jieba
import torch.nn as nn
import random
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

In [7]:
seed = 529
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

## Loading

In [8]:
with open('/content/drive/MyDrive/Colab Notebooks Project/NLP/ESIM/train.json') as f:
    train_raw = f.readlines()
df_train = pd.DataFrame([json.loads(train_raw[x]) for x in range(len(train_raw))])
df_train['label'] = df_train['label'].astype(int)
df_train

,sentence1,sentence2,label
0,蚂蚁借呗等额还款可以换成先息后本吗,借呗有先息到期还本吗,0
1,蚂蚁花呗说我违约一次,蚂蚁花呗违约行为是什么,0
2,帮我看一下本月花呗账单有没有结清,下月花呗账单,0
3,蚂蚁借呗多长时间综合评估一次,借呗得评估多久,0
4,我的花呗账单是***，还款怎么是***,我的花呗，月结出来说让我还***元，我自己算了一下详细名单我应该还***元,1
...,...,...,...
34329,借呗还款日是,借呗还款日计算方法,0
34330,支付宝怎么关闭花呗自动付款功能,如何关闭余额支付功能,0
34331,为什么我借呗利息越来越高了,借呗分期利率怎么计算的,0
34332,二维码开通花呗信用卡付款,怎么样开通花呗信用卡支付码,0


In [9]:
with open('/content/drive/MyDrive/Colab Notebooks Project/NLP/ESIM/dev.json') as f:
    dev_raw = f.readlines()
df_dev = pd.DataFrame([json.loads(dev_raw[x]) for x in range(len(dev_raw))])
df_dev['label'] = df_dev['label'].astype(int)
df_dev

,sentence1,sentence2,label
0,双十一花呗提额在哪,里可以提花呗额度,0
1,花呗支持高铁票支付吗,为什么友付宝不支持花呗付款,0
2,我的蚂蚁花呗支付金额怎么会有限制,我到支付宝实体店消费用花呗支付受金额限制,1
3,为什么有花呗额度不能分期付款,花呗分期额度不足,0
4,赠品不能设置用花呗付款,怎么不能花呗分期付款,0
...,...,...,...
4311,使用的借呗之后，可以出国吗，因为我的工作在国外，我需要在国外还款,借呗这个月分期还款了，还了的钱还可以再次取出用吗,0
4312,还要多久能开通花呗,花呗要多久才会重新开通,0
4313,怎样设置花呗访问手机通讯录,花呗读取不了手机通讯录权限,0
4314,花呗的学历填错了，可不可以改,花呗学历填错了怎样修改,0


In [10]:
with open('/content/drive/MyDrive/Colab Notebooks Project/NLP/ESIM/test.json') as f:
    test_raw = f.readlines()
df_test = pd.DataFrame([json.loads(test_raw[x]) for x in range(len(test_raw))])
df_test

,id,sentence1,sentence2
0,0,借呗什么时候会取消,蚂蚁借呗什么时候可以恢复***个月
1,1,网商贷怎么转变成借呗,如何将网商贷切换为借呗
2,2,我的借呗为啥开通不了,我怎么没法开通借呗
3,3,蚂蚁借呗额度怎么不显示了,蚂蚁借呗额度不显示了
4,4,我的借呗没用给关闭了,把我的借呗关了
...,...,...,...
3856,3856,花呗怎么不能再美团使用了,美团 滴滴用不了花呗
3857,3857,花呗的货币是真实的吗,花呗是自己设定的吗
3858,3858,花呗分期后为什么不可以提前还款,花呗还可以负数么
3859,3859,是用户用花呗支付。我们要收取多少比例的费用,我店铺怎么用不了花呗的


In [11]:
with bz2.open('/content/drive/MyDrive/Colab Notebooks Project/NLP/TouTiao_text/sgns.weibo.word.bz2') as f:
    embedding_raw = f.readlines()

In [12]:
info, *emb = embedding_raw
num, embedding_dim = map(int, info.decode('utf8').split())

word_to_idx, idx_to_word = {'<unk>': 0}, {0: '<unk>'}
embedding_mat = np.zeros((num + 1, embedding_dim))

for idx, i in enumerate(emb):
    word, *embedding = i.decode('utf8').split()
    word_to_idx[word] = idx + 1
    idx_to_word[idx + 1] = word
    embedding_mat[idx] = list(map(float, embedding))

## process context

In [13]:
def transform_to_idx(x):
    idx_list = [word_to_idx[i] if i in word_to_idx else 0 for i in jieba.lcut(x)]
    length = len(idx_list)
    if length >= 30:
        return np.array(idx_list[:30])
    else:
        return np.array(idx_list + [0]*(30-length))

for df in [df_train, df_dev, df_test]:
    df['query'] = df['sentence1'].apply(transform_to_idx)
    df['doc'] = df['sentence2'].apply(transform_to_idx)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.908 seconds.
Prefix dict has been built successfully.


## dataset

In [14]:
class MyDataSet(Dataset):
    def __init__(self, query, doc, y=None):
        super().__init__()
        self.x = [(torch.LongTensor(q), torch.LongTensor(d)) for q, d in zip(query, doc)]
        self.y = torch.LongTensor(y) if y is not None else [0] * len(query) 

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

train_set = MyDataSet(df_train['query'], df_train['doc'], df_train['label'])
dev_set = MyDataSet(df_dev['query'], df_dev['doc'], df_dev['label'])
test_set = MyDataSet(df_test['query'], df_test['doc'])

# 加载 DataLoader
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
dev_loader = DataLoader(dev_set, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=128, shuffle=True, num_workers=2)

## Backbone

In [15]:
config = {
    'embedding_mat': torch.Tensor(embedding_mat),  # 词向量矩阵
    'is_embedding_freeze': True,                   # 词向量是否进行固定
    'base_rnn': nn.GRU,                            # BiLSTM 使用的 rnn 类型
    'stack_rnn_layers': 2,                         # BiLSTM stack 的层数
    'input_size': embedding_dim,                   # Input Encoding 的 BiLSTM 初始输入的特征维度
    'hidden_size': 32,                             # BiLSTM 隐藏层的特征维度
    'rnn_layer': 1,                                # 每个 BiLSTM 里的层数
    'mlp_dim': 32,                                 # 1 层分类层的维度
    'num_label': 2                                 # 标签值个数
}

### Embedding

In [16]:
class EmbeddingPair(nn.Module):
    def __init__(self, confing):
        super(EmbeddingPair, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(
            embeddings=config['embedding_mat'], 
            freeze=config['is_embedding_freeze']
            )

    def forward(self, pair_x):
        query, doc = pair_x
        query_embedding = self.embedding(query)
        doc_embedding = self.embedding(doc)
        return query_embedding, doc_embedding

### BiLSTM

In [17]:
class BiLSTM(nn.Module):
    def __init__(self, config, layer_name):
        super(BiLSTM, self).__init__()

        # Input Encoding 与 Composition Layer 的输入维度不同
        if layer_name == 'encoding':
            input_size = config['input_size']
        elif layer_name == 'composition':
            input_size = config['hidden_size']
        
        # stack rnn 层数
        self.layers = config['stack_rnn_layers']
        
        # 两层堆叠的 RNN 层，前一层的 output 是后一层的 input
        self.stack_rnn = nn.ModuleList()
        
        for i in range(config['stack_rnn_layers']):
            rnn_input_size = input_size if i == 0 else 2 * config['hidden_size']

            self.stack_rnn.append(
                config['base_rnn'](
                    input_size=rnn_input_size,
                    hidden_size=config['hidden_size'],
                    num_layers=config['rnn_layer'],
                    bidirectional=True
                    ))

    def forward(self, pair_x):
        query, doc = pair_x

        # (batch, seq, embedding_dim) - tensor.transpose -> (seq, batch，input_size)
        # LSTM 的输入需要维度 (seq, batch，input_size)
        # 初始值当作最开始的输出值
        query = query.transpose(1,0)
        doc = doc.transpose(1,0)

        # 每一层 stackrnn 计算
        for i in range(len(self.stack_rnn)):
            query, _ = self.stack_rnn[i](query)  # Outputs: output, (h_n, c_n)
            doc, _ = self.stack_rnn[i](doc)  # Outputs: output, (h_n, c_n)

        # 维度转换 (seq, batch, hidden_dim) -> (batch, seq, hidden_dim) 输出
        query = query.permute(1, 0, 2)
        doc = doc.permute(1, 0, 2)
        
        return query, doc

### EnhanceInformation

In [18]:
class EnhanceInformation(nn.Module):
    def __init__(self):
        super(EnhanceInformation, self).__init__()

    def forward(self, pair_x):
        
        # (batch, seq, hidden_dim)
        query, doc = pair_x

        # 求出对应相似度矩阵 e -> (batch, seq, seq)
        e = torch.bmm(query, doc.transpose(1, 2))  # Equation (11) 

        # 求出对应 query 和 doc 的 attention weight -> (batch, seq, seq)
        attention_query_doc = F.softmax(e, dim=2)  # Equation (12) 
        attention_doc_query = F.softmax(e, dim=1)  # Equation (13)

        # 求 attention 加权求和后的矩阵 -> (batch, seq, hidden_dim)
        query_attention = torch.bmm(attention_query_doc, doc)
        doc_attention = torch.bmm(attention_doc_query, query)

        # Enhancement of local inference information -> (batch, seq, 4 * hidden_dim)
        m_query = torch.cat((query, query_attention, query-query_attention, query*query_attention), dim=-1)  # Equation (14)
        m_doc = torch.cat((doc, doc_attention, doc-doc_attention, doc*doc_attention), dim=-1) # Equation (15)

        return m_query, m_doc

### FeedForward

In [19]:
class ForwardPair(nn.Module):
    def __init__(self, config):
        super(ForwardPair, self).__init__()
        self.linear = nn.Linear(config['hidden_size']*2*4, config['hidden_size'])
        self.relu = nn.ReLU()

    def forward(self, pair_x):
        query, doc = pair_x
        query = self.relu(self.linear(query))
        doc = self.relu(self.linear(doc))
        return query, doc

### Pooling

In [20]:
class Pooling(nn.Module):
    def __init__(self):
        super(Pooling, self).__init__()

    def forward(self, pair_x):
        query, doc = pair_x

        # Equation (18)
        query_ave = query.sum(dim=1) / query.shape[1]
        query_max, _ = query.max(dim=1)

        # Equation (19)
        doc_ave = doc.sum(dim=1) / doc.shape[1]
        doc_max, _ = doc.max(dim=1)

        # 合并成一个
        v = torch.cat((query_ave, query_max, doc_ave, doc_max), dim=-1)
        
        return v

### Classification

In [21]:
class Classification(nn.Module):
    def __init__(self, config):
        super(Classification, self).__init__()
        self.mlp = nn.Linear(config['hidden_size']*2*4, config['mlp_dim'])
        self.tanh = nn.Tanh()
        self.out = nn.Linear(config['mlp_dim'], config['num_label'])

    def forward(self, x):
        x = self.mlp(x)
        x = self.tanh(x)
        x = self.out(x)
        return x

### ESIM Model

In [22]:
class ESIM(nn.Module):
    def __init__(self, config):
        super(ESIM, self).__init__()
        self.embedding = EmbeddingPair(config)
        self.input_encoder = BiLSTM(config, 'encoding')
        self.enhance_info = EnhanceInformation()
        self.forward_pair = ForwardPair(config)
        self.composition_layer = BiLSTM(config, 'composition')
        self.pooling = Pooling()
        self.clf = Classification(config)
        
    def forward(self, pair_x):
        pair_x = self.embedding(pair_x)
        pair_x = self.input_encoder(pair_x)
        pair_x = self.enhance_info(pair_x)
        pair_x = self.forward_pair(pair_x)
        pair_x = self.composition_layer(pair_x)
        x = self.pooling(pair_x)
        output = self.clf(x)

        return output

## Train Model

In [23]:
class TrainModel(pl.LightningModule):
    def __init__(self, network):
        super(TrainModel, self).__init__()
        self.backbone = network
        self.loss = nn.CrossEntropyLoss()
        self.lr = 1e-6

    def training_step(self, batch, batch_idx):
        x, y = batch
        output = self.backbone(x)
        loss = self.loss(output, y)
        self.log('Training Loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self.backbone(x)
        loss = self.loss(output, y)
        self.log('Validation Loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def forward(self, x):
        predict = self.backbone(x)
        predict = F.softmax(predict, dim=-1).argmax(dim=-1)
        return predict

In [24]:
%load_ext tensorboard

## Trainning

In [26]:
tb_logger = pl_loggers.TensorBoardLogger('logs/')

esim = ESIM(config)
model = TrainModel(esim)
trainer = pl.Trainer(max_epochs=10, logger=tb_logger, gpus=1)
trainer.fit(model, train_loader, dev_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type             | Params
----------------------------------------------
0 | backbone | ESIM             | 58.7 M
1 | loss     | CrossEntropyLoss | 0     
----------------------------------------------
130 K     Trainable params
58.6 M    Non-trainable params
58.7 M    Total params
234.767   Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:373: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  f"Your {mode}_dataloader has `shuffle=True`, it is best practice to turn"


In [28]:
# %tensorboard --logdir logs # colab 打开